# Tarea 3: Derivación automática 1

Fecha de envío del PR inicial: **lunes 27 de marzo, antes de la clase**

Fecha de aceptación del PR: **martes 4 de abril, antes de la clase**

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que llamaremos `Dual` y que defina los duales, donde el parámetro debe ser un subtipo de `Real`; la siguiente celda sirve para empezar. La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

---

In [1]:
"""
    Dual{T<:Real}

Definición de los duales, donde lo campos son:
"""
type Dual{T<:Real}
    fun::T
    der::T
end

Dual

In [2]:
#= 
Definan un método que permita la promoción automática cuando 
las entradas para definir el dual no son del mismo tipo
=#
"""
promoviendo los duales del tipo distinto
"""
function Dual(fun,der)
 return Dual(promote(fun,der)...)
end

Dual

In [3]:
#= 
Aquí se define un método que garantiza que el dual de una constante 
(número) cumple lo requerido
=#
function Dual(a::Real)
    return Dual(a,0.0)
end

Dual{T<:Real}

In [4]:
#= 
Aquí se define la función `xdual`, que se usará para identificar 
la variable independiente. La función dependerá de x_0, y debe 
regresar el Dual apropiado a la variable independiente
=#

"""
    xdual(x0) -> Dual(x0, 1)

...
"""
function xdual(x0)
    return Dual(x0,1)
end

xdual

In [5]:
#= 
Muestren que su código funciona con tests adecuados para crear duales,
para promoverlos, y al definir el dual de un número y `xdual`.

En esto es útil usar la infraestructura de Julia; ver:
https://julia.readthedocs.io/en/stable/stdlib/test/

using Base.Test

a = Dual(1, 2.0)
@test a.fun == 1.0
@test a.der == 2.0

=#
using Base.Test
a=Dual(1,2.0)
@test a.fun==1
@test a.der==2

b=Dual(3)
@test b.fun==3
@test b.der==0


c=xdual(10)
@test c.fun==10
@test c.der==1


Test Passed
  Expression: c.der == 1
   Evaluated: 1 == 1

---

## Ejercicio 2

Implementen *todas* las operaciones aritméticas definidas arriba. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`), como por ejemplo `a+b` o `b+a`, etc. Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). 

Implementen también la comparación entre duales (`==`). 

Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [15]:
import Base: +, -, *, /, ^, ==

#= 
Aquí se implementan los métodos necesarios para cada función; 
en el caso de ^ por ahora nos conformaremos con que la potencia 
sea entera.
=#
+(A::Dual,B::Dual)=Dual(A.fun+B.fun,A.der+B.der)
-(A::Dual,B::Dual)=Dual(A.fun-B.fun,A.der-B.der)
*(A::Dual,B::Dual)=Dual(A.fun*B.fun,(A.der*B.fun)+(A.fun*B.der))
/(A::Dual,B::Dual)=Dual((A.fun)/(B.fun),((A.fun*B.der)-(A.der*B.fun))/(B.fun)^2)
^(A::Dual,beta::Real)=Dual((A.fun)^(beta),beta*((A.fun)^(beta-1))*(A.der))
==(A::Dual,B::Dual)=Bool(A.fun==B.fun && A.der==B.der)


== (generic function with 119 methods)

, Main.Dual) in module Main at In[9]:8 overwritten at In[15]:8.


In [19]:
# Aquí se incluyen las pruebas necesarias
using Base.Test
alfa=Dual(5)+Dual(5)
@test alfa.fun==10
@test alfa.der==0

betha=Dual(5)-Dual(5)
@test betha.fun==0
@test betha.der==0

gamma=Dual(5)*Dual(6)
@test gamma.fun==30
@test gamma.der==0

delta=Dual(6)/Dual(2)
@test delta.fun==3
@test delta.der==0

epsilon=Dual(2)^4.0
@test epsilon.fun==16
@test epsilon.der==0



Test Passed
  Expression: epsilon.der == 0
   Evaluated: 0.0 == 0

In [20]:
zeta=(Dual(12)==Dual(12))

true

---

## Ejercicio 3

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo cuyo nombre será "AutomDiff.jl" y cuya estructura será la siguiente

```julia
#=
Aquí viene una explicación de lo que se hace en el módulo, 
los autores y la fecha
=#

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    ...

end
```

Todas las pruebas deberán ser incluidas en un archivo separado llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos
...

```

Estos dos archivos deben incluirlos en el envío de su tarea (además del archivo `Tarea3.ipynb`).